In [90]:
#params = []
#params.append(["train_size", "K", "learn_rate", "iterations", "w", "error"])

In [91]:
import numpy as np
import csv
import math

train_file = 'train.csv'
test_file  = 'test.csv'
soln_file  = 'user_median.csv'

train_size = .9
K = 100
learn_rate = 0.000001
iterations = 1
w = .2
weight_user_median = .9
penalty = .02


#load CSV
train_data = {}
test_data = {}
ndata = 0
with open(train_file, 'r') as train_fh:
    ndata = sum(1 for row in train_fh) - 1
    
splitting = np.random.binomial(1, train_size, size=ndata)


In [92]:

with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    
    for i, row, in enumerate(train_csv):
        user   = row[0]
        artist = row[1]
        plays  = row[2]

        if splitting[i]: 
            if not user in train_data:
                train_data[user] = {}
            train_data[user][artist] = int(plays)
            
        else:
            if not user in test_data:
                test_data[user] = {}

            test_data[user][artist] = int(plays)
              
# Compute the global median and per-user median.
plays_array  = []
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

In [93]:
user_feats = {}
artist_feats = {}


init = np.array(K * [math.sqrt(float(global_median)/K)])

artist_plays = {}
for user, user_data in train_data.iteritems():
    if not user in user_feats:
        user_feats[user] = init
    for artist, plays in user_data.iteritems():
        if not artist in artist_plays:
            artist_plays[artist] = []
        artist_plays[artist].append(plays)
        if not artist in artist_feats:
            artist_feats[artist] = init
 

In [94]:
artist_meds = {}
for artist in artist_plays.keys():
    artist_meds[artist] = np.median(np.array(artist_plays[artist]))

 

In [95]:
i = 7
test_artist = artist_feats.keys()[i]

print artist_meds[test_artist]


209.0


In [96]:
weight_user_median = .9

def predict_plays(user, artist):
    if (not user in user_feats) or (not artist in artist_feats):
        return global_median
    else:
        return np.dot(np.array(user_feats[user]), np.array(artist_feats[artist]))

    
def modify_prediction(user, artist, prediction):
    user_bias = user_medians[user] - global_median
    return w*prediction + (1-w)*(artist_meds[artist] + user_bias)

def modify_prediction2(user, artist, prediction):
    return (1 - weight_user_median)*prediction + weight_user_median*(user_medians[user])

def train(data):
    i = 0
    
    for k in range(iterations):
        for j, (user, user_data) in enumerate(train_data.iteritems()):
            if j % 10000 == 0:
                print "\r", j,k,

            for artist, plays in user_data.iteritems(): 
                
                pred = predict_plays(user,artist)
                error = int(plays) - pred

                artist_feats[artist] += learn_rate*(error*user_feats[user] - penalty * artist_feats[artist])
                user_feats[user] +=  learn_rate*(error*artist_feats[artist] - penalty * user_feats[user])

train(train_data)

230000 0


In [97]:
print user_medians[user_feats.keys()[1]]



def calc_error(data):
    errors = []
    pred = 0
    for j, (user, user_data) in enumerate(data.iteritems()):
        if j % 1000 == 0:
            print "\r", j,
        for artist, plays in user_data.iteritems():
                pred = predict_plays(user,artist)
                
                newpred = modify_prediction(user,artist,pred)
                error = abs(int(plays) - newpred)
                errors.append(error)
               
                
    return sum(np.array(errors))/len(errors)
                
error = calc_error(test_data)

print "Error: ", error

361.0
193000 Error:  143.223388553


In [102]:

def calc_error(data):
    errors = []
    errors2 = []
    pred = 0
    for j, (user, user_data) in enumerate(data.iteritems()):
        if j % 10000 == 0:
            print "\r", j,
        for artist, plays in user_data.iteritems():
            pred = predict_plays(user,artist)
                
            newpred = modify_prediction(user,artist,pred)
            error = abs(int(plays) - newpred)
            errors.append(error)

            newpred2 = modify_prediction2(user,artist,pred)
            error2 = abs(int(plays) - newpred2)
            errors2.append(error2)


    return (sum(np.array(errors))/len(errors), sum(np.array(errors2))/len(errors2))
        
error = calc_error(test_data)

print "Error: ", error


190000 Error:  (143.22338855320115, 139.53333622088593)


In [103]:
params.append([train_size, K, learn_rate, iterations, w, weight_user_median, penalty, error])

In [104]:
print params

[['train_size', 'K', 'learn_rate', 'iterations', 'w', 'error'], [0.9, 100, 1e-06, 10, 0.1, (141.20002311812974, 140.09672290114969)], [0.9, 100, 1e-05, 10, 0.1, (141.93318932550011, 140.61400537502971)], [0.9, 100, 0.0001, 10, 0.1, (nan, nan)], [0.9, 100, 1e-06, 10, 0.1, (140.5000839421362, 139.19937042024716)], [0.9, 100, 1e-06, 10, 0.1, (140.81387087540361, 139.51881098451238)], [0.9, 100, 1e-06, 10, 0.1, 0.03, (140.81387087540361, 139.51881098451238)], [0.9, 100, 1e-06, 1, 0.2, 0.9, 0.02, 143.22338855320115], [0.9, 100, 1e-06, 1, 0.2, 0.9, 0.02, (143.22338855320115, 139.53333622088593)]]
